In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Dataset inladen (geef hier het juiste pad op)
df = pd.read_csv('SAIL_Amsterdam_10min_Weather_2025-08-20_to_2025-08-24.csv', names=['time', 'temperature', 'humidity', 'rain'], header=0, parse_dates=['time'])

# Rename columns to match expected names
df = df.rename(columns={
    'temperature': 'temperature_2m (°C)',
    'humidity': 'relativehumidity_2m (%)',
    'rain': 'precipitation (mm)'
})
# Add dummy latitude, longitude, and wind columns for compatibility
df['latitude'] = 52.41
df['longitude'] = 4.84
df['wind_speed_10m (km/h)'] = 10.0  # dummy wind data

# 2. Convert 'time' naar datetime en instellen als index
df.set_index('time', inplace=True)

# 3. Voorbeeld: dagelijkse samenvatting per locatie (gemiddelde, minimum, maximum)
daily_summary = df.groupby([pd.Grouper(freq='D'), 'latitude', 'longitude']).agg(
    temp_mean=('temperature_2m (°C)', 'mean'),
    temp_min=('temperature_2m (°C)', 'min'),
    temp_max=('temperature_2m (°C)', 'max'),
    humidity_mean=('relativehumidity_2m (%)', 'mean'),
    precipitation_sum=('precipitation (mm)', 'sum'),
    wind_mean=('wind_speed_10m (km/h)', 'mean')
)

print("Dagelijkse samenvatting per locatie:")
print(daily_summary)

# 4. Voorbeeld: 10-minuten samenvatting (hier voor locatie 52.41°N, 4.84°E)
mask = (df['latitude'] == 52.41) & (df['longitude'] == 4.84)
ten_min = df[mask].resample('10min').agg(
    temperature=('temperature_2m (°C)', 'mean'),
    humidity=('relativehumidity_2m (%)', 'mean'),
    precipitation=('precipitation (mm)', 'sum'),
    wind=('wind_speed_10m (km/h)', 'mean')
)

print("\n10-minuten samenvatting (locatie 52.41N 4.84E):")
print(ten_min.head())

# 5. Grafiek van temperatuur over tijd
plt.figure(figsize=(12, 6))
plt.plot(ten_min.index, ten_min['temperature'], label='Temperature (°C)')
plt.title('Temperature over Time (10-min intervals)')
plt.xlabel('Time')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.grid(True)
plt.show()

# 6. Lijn voor luchtvochtigheid
plt.figure(figsize=(12, 6))
plt.plot(ten_min.index, ten_min['humidity'], label='Humidity (%)', color='orange')
plt.title('Humidity over Time (10-min intervals)')
plt.xlabel('Time')
plt.ylabel('Humidity (%)')
plt.legend()
plt.grid(True)
plt.show()

# 7. Lijn voor neerslag
plt.figure(figsize=(12, 6))
plt.plot(ten_min.index, ten_min['precipitation'], label='Precipitation (mm)', color='blue')
plt.title('Precipitation over Time (10-min intervals)')
plt.xlabel('Time')
plt.ylabel('Precipitation (mm)')
plt.legend()
plt.grid(True)
plt.show()

# 8. Lijn voor windsnelheid
plt.figure(figsize=(12, 6))
plt.plot(ten_min.index, ten_min['wind'], label='Wind Speed (km/h)', color='green')
plt.title('Wind Speed over Time (10-min intervals)')
plt.xlabel('Time')
plt.ylabel('Wind Speed (km/h)')
plt.legend()
plt.grid(True)
plt.show()

# 9. Subplot voor alle variabelen
fig, axs = plt.subplots(4, 1, figsize=(12, 16), sharex=True)
axs[0].plot(ten_min.index, ten_min['temperature'], color='red')
axs[0].set_title('Temperature (°C)')
axs[0].grid(True)

axs[1].plot(ten_min.index, ten_min['humidity'], color='orange')
axs[1].set_title('Humidity (%)')
axs[1].grid(True)

axs[2].plot(ten_min.index, ten_min['precipitation'], color='blue')
axs[2].set_title('Precipitation (mm)')
axs[2].grid(True)

axs[3].plot(ten_min.index, ten_min['wind'], color='green')
axs[3].set_title('Wind Speed (km/h)')
axs[3].grid(True)

plt.xlabel('Time')
plt.tight_layout()
plt.show()